In [108]:
import kagglehub
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [109]:
# Download latest version
path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\reminovici\.cache\kagglehub\datasets\nehalbirla\vehicle-dataset-from-cardekho\versions\4


In [110]:
os.listdir('C:/Users/reminovici/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4')

['car data.csv',
 'CAR DETAILS FROM CAR DEKHO.csv',
 'Car details v3.csv',
 'car details v4.csv']

In [111]:
import os
csv_path = os.path.join(path, 'car details v4.csv')
df = pd.read_csv(csv_path)
df

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,Mahindra,XUV500 W8 [2015-2017],850000,2016,90300,Diesel,Manual,Surat,White,First,Individual,2179 cc,138 bhp @ 3750 rpm,330 Nm @ 1600 rpm,FWD,4585.0,1890.0,1785.0,7.0,70.0
2055,Hyundai,Eon D-Lite +,275000,2014,83000,Petrol,Manual,Ahmedabad,White,Second,Individual,814 cc,55 bhp @ 5500 rpm,75 Nm @ 4000 rpm,FWD,3495.0,1550.0,1500.0,5.0,32.0
2056,Ford,Figo Duratec Petrol ZXI 1.2,240000,2013,73000,Petrol,Manual,Thane,Silver,First,Individual,1196 cc,70 bhp @ 6250 rpm,102 Nm @ 4000 rpm,FWD,3795.0,1680.0,1427.0,5.0,45.0
2057,BMW,5-Series 520d Luxury Line [2017-2019],4290000,2018,60474,Diesel,Automatic,Coimbatore,White,First,Individual,1995 cc,188 bhp @ 4000 rpm,400 Nm @ 1750 rpm,RWD,4936.0,1868.0,1479.0,5.0,65.0


In [112]:
X = df.drop(columns=['Make', 'Model', 'Price'])
y = df['Price']

y

0        505000
1        450000
2        220000
3        799000
4       1950000
         ...   
2054     850000
2055     275000
2056     240000
2057    4290000
2058     670000
Name: Price, Length: 2059, dtype: int64

In [113]:
X.isnull().sum()

Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 80
Max Power              80
Max Torque             80
Drivetrain            136
Length                 64
Width                  64
Height                 64
Seating Capacity       64
Fuel Tank Capacity    113
dtype: int64

In [114]:
import re

# Extract max power (bhp) and rpm
X['max_power_bhp'] = X['Max Power'].str.extract(r'(\d+\.?\d*)').astype(float)
X['max_power_rpm'] = X['Max Power'].str.extract(r'@ (\d+)').astype(float)

# Extract max torque (Nm) and rpm
X['max_torque_nm'] = X['Max Torque'].str.extract(r'(\d+\.?\d*)').astype(float)
X['max_torque_rpm'] = X['Max Torque'].str.extract(r'@ (\d+)').astype(float)


X = X.drop(columns=['Max Power', 'Max Torque'])
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Year                2059 non-null   int64  
 1   Kilometer           2059 non-null   int64  
 2   Fuel Type           2059 non-null   object 
 3   Transmission        2059 non-null   object 
 4   Location            2059 non-null   object 
 5   Color               2059 non-null   object 
 6   Owner               2059 non-null   object 
 7   Seller Type         2059 non-null   object 
 8   Engine              1979 non-null   object 
 9   Drivetrain          1923 non-null   object 
 10  Length              1995 non-null   float64
 11  Width               1995 non-null   float64
 12  Height              1995 non-null   float64
 13  Seating Capacity    1995 non-null   float64
 14  Fuel Tank Capacity  1946 non-null   float64
 15  max_power_bhp       1979 non-null   float64
 16  max_po

In [115]:
X['Engine'] = X['Engine'].str.split().str[0].astype(float)
X

,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,max_power_bhp,max_power_rpm,max_torque_nm,max_torque_rpm
0,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198.0,FWD,3990.0,1680.0,1505.0,5.0,35.0,87.0,6000.0,109.0000,4500.0
1,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248.0,FWD,3995.0,1695.0,1555.0,5.0,42.0,74.0,4000.0,190.0000,2000.0
2,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197.0,FWD,3585.0,1595.0,1550.0,5.0,35.0,79.0,6000.0,112.7619,4000.0
3,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197.0,FWD,3995.0,1745.0,1510.0,5.0,37.0,82.0,6000.0,113.0000,4200.0
4,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393.0,RWD,4735.0,1830.0,1795.0,7.0,55.0,148.0,3400.0,343.0000,1400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,2016,90300,Diesel,Manual,Surat,White,First,Individual,2179.0,FWD,4585.0,1890.0,1785.0,7.0,70.0,138.0,3750.0,330.0000,1600.0
2055,2014,83000,Petrol,Manual,Ahmedabad,White,Second,Individual,814.0,FWD,3495.0,1550.0,1500.0,5.0,32.0,55.0,5500.0,75.0000,4000.0
2056,2013,73000,Petrol,Manual,Thane,Silver,First,Individual,1196.0,FWD,3795.0,1680.0,1427.0,5.0,45.0,70.0,6250.0,102.0000,4000.0
2057,2018,60474,Diesel,Automatic,Coimbatore,White,First,Individual,1995.0,RWD,4936.0,1868.0,1479.0,5.0,65.0,188.0,4000.0,400.0000,1750.0


In [116]:
X.isnull().sum()

Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 80
Drivetrain            136
Length                 64
Width                  64
Height                 64
Seating Capacity       64
Fuel Tank Capacity    113
max_power_bhp          80
max_power_rpm         212
max_torque_nm          80
max_torque_rpm        208
dtype: int64

In [117]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Year                2059 non-null   int64  
 1   Kilometer           2059 non-null   int64  
 2   Fuel Type           2059 non-null   object 
 3   Transmission        2059 non-null   object 
 4   Location            2059 non-null   object 
 5   Color               2059 non-null   object 
 6   Owner               2059 non-null   object 
 7   Seller Type         2059 non-null   object 
 8   Engine              1979 non-null   float64
 9   Drivetrain          1923 non-null   object 
 10  Length              1995 non-null   float64
 11  Width               1995 non-null   float64
 12  Height              1995 non-null   float64
 13  Seating Capacity    1995 non-null   float64
 14  Fuel Tank Capacity  1946 non-null   float64
 15  max_power_bhp       1979 non-null   float64
 16  max_po

In [118]:
imputer = SimpleImputer(strategy='mean')
X_copy = X

X_copy[['Engine', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity', 'max_power_bhp', 'max_power_rpm', 'max_torque_nm', 'max_torque_rpm']] = imputer.fit_transform(X_copy[['Engine', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity', 'max_power_bhp', 'max_power_rpm', 'max_torque_nm', 'max_torque_rpm']])
X_copy

X = X_copy

In [119]:
com = X['Drivetrain'].mode()[0]
X['Drivetrain'] = X['Drivetrain'].fillna(com)


X.isnull().sum()

Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
max_power_bhp         0
max_power_rpm         0
max_torque_nm         0
max_torque_rpm        0
dtype: int64

In [120]:
X

,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,max_power_bhp,max_power_rpm,max_torque_nm,max_torque_rpm
0,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198.0,FWD,3990.0,1680.0,1505.0,5.0,35.00000,87.0,6000.0,109.0000,4500.0
1,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248.0,FWD,3995.0,1695.0,1555.0,5.0,42.00000,74.0,4000.0,190.0000,2000.0
2,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197.0,FWD,3585.0,1595.0,1550.0,5.0,35.00000,79.0,6000.0,112.7619,4000.0
3,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197.0,FWD,3995.0,1745.0,1510.0,5.0,37.00000,82.0,6000.0,113.0000,4200.0
4,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393.0,RWD,4735.0,1830.0,1795.0,7.0,55.00000,148.0,3400.0,343.0000,1400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,2016,90300,Diesel,Manual,Surat,White,First,Individual,2179.0,FWD,4585.0,1890.0,1785.0,7.0,70.00000,138.0,3750.0,330.0000,1600.0
2055,2014,83000,Petrol,Manual,Ahmedabad,White,Second,Individual,814.0,FWD,3495.0,1550.0,1500.0,5.0,32.00000,55.0,5500.0,75.0000,4000.0
2056,2013,73000,Petrol,Manual,Thane,Silver,First,Individual,1196.0,FWD,3795.0,1680.0,1427.0,5.0,45.00000,70.0,6250.0,102.0000,4000.0
2057,2018,60474,Diesel,Automatic,Coimbatore,White,First,Individual,1995.0,RWD,4936.0,1868.0,1479.0,5.0,65.00000,188.0,4000.0,400.0000,1750.0


In [121]:

ct = ColumnTransformer(
    transformers=[('enc', OneHotEncoder(), [2, 3, 4, 5, 6, 7, 9])]
)

X_enc = ct.fit_transform(X)
X_dense = X_enc.toarray()
X = pd.DataFrame(X_dense)

X

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2055,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2056,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2057,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [122]:
X = np.array(X)
X

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.]], shape=(2059, 117))

In [123]:
# add bias column (intercept column)

X_with_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
X = X_with_intercept
X

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]], shape=(2059, 118))

In [124]:
y = np.array(y)
type(y)

numpy.ndarray

In [125]:
#scaling

sc1 = StandardScaler()

X = sc1.fit_transform(X)
X


array([[ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
         0.63600475, -0.42976162],
       [ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
         0.63600475, -0.42976162],
       [ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
         0.63600475, -0.42976162],
       ...,
       [ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
         0.63600475, -0.42976162],
       [ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
        -1.57231529,  2.32687134],
       [ 0.        , -0.15775932, -0.02204334, ..., -0.39014152,
        -1.57231529,  2.32687134]], shape=(2059, 118))

In [126]:
y = sc1.fit_transform(y.reshape(-1, 1))
y

array([[-0.49518257],
       [-0.51791649],
       [-0.61298559],
       ...,
       [-0.60471871],
       [ 1.06932413],
       [-0.42698083]], shape=(2059, 1))

In [127]:
# splitting the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
# use stochastic gradient descent to minimize the loss function
# use normal eqautions (later)

In [ ]:
# generate weights

weights = np.zeros((X_train.shape[1], 1))
weights

In [130]:
''' 
MAE as loss function

w -= alpha * gradient

patience = how may epochs without mae improvment will the method support
 
 '''


def fit(X_train, y_train, weights, alpha, epochs, patience=30):
    n = X_train.shape[0]
    best_mae = float('inf')
    epochs_without_mae_improv = 0

    for epoch in range(epochs):
        # iterate through all examples
        total_loss = 0
        for i in range(n):
            xi = X_train[i].reshape(-1, 1)
            yi = y_train[i]

            prediction = np.dot(xi.T, weights)

            #compute error (residual)
            error = prediction - yi

            #compute gradient -> see derivative of MAE
            gradient = np.sign(error) * xi

            #update weights
            weights -= alpha * gradient

            #gather total loss per epoch to monitor performance
            total_loss += np.abs(error)

        mae = float(total_loss / n)

        if mae < best_mae:
            best_mae = mae
            epochs_without_mae_improv = 0
        else:
            epochs_without_mae_improv += 1

        if epochs_without_mae_improv >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break

        print(f"MAE for epoch {epoch + 1}, MAE: {mae: .4f}")

    return weights





In [131]:
# now predict on test data
'''
uses the trained weights to make a prediction on an unseen examples

X = unseen examples
y = actual target
weights = trained weights

'''
def predict(X, y, weights):
    n = X.shape[0]
    predictions = []
    absolute_losses = []

    y_flat = y.flatten() if len(y.shape) > 1 else y

    for i in range(n):
        xi = X[i]
        yi = y[i]

        prediction = np.dot(xi, weights).item()

        loss = prediction - yi
    
        predictions.append(prediction)
        absolute_losses.append(abs(loss))

    validation_mae = np.mean(absolute_losses)
    predictionsDf = pd.DataFrame({'actual': y_flat, 'predicted': predictions, 'absolute_error': absolute_losses})
    return predictionsDf, validation_mae

In [132]:
weights = fit(X_train, y_train, weights, 0.00001, 1000)


C:\Users\reminovici\AppData\Local\Temp\ipykernel_3140\3937910892.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mae = float(total_loss / n)


MAE for epoch 1, MAE:  0.5790
MAE for epoch 2, MAE:  0.5560
MAE for epoch 3, MAE:  0.5350
MAE for epoch 4, MAE:  0.5152
MAE for epoch 5, MAE:  0.4966
MAE for epoch 6, MAE:  0.4795
MAE for epoch 7, MAE:  0.4639
MAE for epoch 8, MAE:  0.4497
MAE for epoch 9, MAE:  0.4369
MAE for epoch 10, MAE:  0.4252
MAE for epoch 11, MAE:  0.4146
MAE for epoch 12, MAE:  0.4052
MAE for epoch 13, MAE:  0.3971
MAE for epoch 14, MAE:  0.3903
MAE for epoch 15, MAE:  0.3850
MAE for epoch 16, MAE:  0.3809
MAE for epoch 17, MAE:  0.3776
MAE for epoch 18, MAE:  0.3751
MAE for epoch 19, MAE:  0.3731
MAE for epoch 20, MAE:  0.3713
MAE for epoch 21, MAE:  0.3697
MAE for epoch 22, MAE:  0.3683
MAE for epoch 23, MAE:  0.3671
MAE for epoch 24, MAE:  0.3660
MAE for epoch 25, MAE:  0.3651
MAE for epoch 26, MAE:  0.3643
MAE for epoch 27, MAE:  0.3634
MAE for epoch 28, MAE:  0.3628
MAE for epoch 29, MAE:  0.3621
MAE for epoch 30, MAE:  0.3614
MAE for epoch 31, MAE:  0.3608
MAE for epoch 32, MAE:  0.3604
MAE for epoch 33,

In [133]:
preds, mae = predict(X_test, y_test, weights)
preds

,actual,predicted,absolute_error
0,1.280130,1.118251,[0.16187888088305646]
1,-0.362913,-0.420149,[0.05723655627421337]
2,-0.416647,-0.345271,[0.07137626559767635]
3,-0.311245,-0.377748,[0.06650337577579446]
4,-0.364980,-0.394844,[0.029864327451032402]
...,...,...,...
407,-0.590252,-0.514056,[0.0761953729062057]
408,-0.063238,0.458374,[0.5216124888786469]
409,1.606671,0.156916,[1.4497551838143359]
410,-0.569584,-0.500611,[0.06897391261766372]


In [136]:
print(f'Validation MAE: {mae}')

Validation MAE: 0.3823351103690121


In [137]:
# Normal equations instead of SGD => brute force optimal weights

In [139]:
weights2 = np.linalg.pinv(X.T @ X) @ (X.T @ y)

In [140]:
preds2, mae2 = predict(X, y, weights2)
preds2

,actual,predicted,absolute_error
0,-0.495183,-0.067890,[0.42729291196318797]
1,-0.517916,-0.756930,[0.23901393869554843]
2,-0.612986,-0.429753,[0.1832330604160448]
3,-0.373659,-0.470783,[0.09712374070515023]
4,0.102099,0.145707,[0.04360728291499147]
...,...,...,...
2054,-0.352579,-0.366482,[0.013902813569179606]
2055,-0.590252,-0.751290,[0.16103798516001488]
2056,-0.604719,-0.775665,[0.17094670143311275]
2057,1.069324,0.680149,[0.3891746393101607]


In [141]:
print(f'Validation MAE: {mae2}')

Validation MAE: 0.39013850955131335
